In [1]:
"""
The helper file
"""

import os
import pickle
import copy
import numpy as np


CODES = {'<unk>': 0, '<s>': 1, '</s>': 2}
# Extra vocabulary symbols
sos = '<s>'
eos = '</s>' # also function as PAD
unk = '<unk>'

extra_tokens = [unk, sos, eos]

start_token = extra_tokens.index(sos) # start_token = 0
end_token = extra_tokens.index(eos)   # end_token = 1
unk_token = extra_tokens.index(unk)

def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8') as f:
        data = f.read()

    return data

def preprocess_and_save_data(source_path, target_path):
    """
    Preprocess Text Data.  Save to to file.
    """
    
    # Preprocess
    source_text = load_data(source_path)
    target_text = load_data(target_path)

    source_text = source_text.lower()
    target_text = target_text.lower()

    source_vocab_to_int, source_int_to_vocab = create_lookup_tables(source_text)
    
    target_vocab_to_int, target_int_to_vocab = create_lookup_tables(target_text)

    print(target_int_to_vocab)
    
    source_text, target_text = text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int)

    # Save Data
    pickle.dump((
        (source_text, target_text),
        (source_vocab_to_int, target_vocab_to_int),
        (source_int_to_vocab, target_int_to_vocab)), open('preprocess.p', 'wb'))


def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    """
    vocab = set(text.split())
    vocab_to_int = copy.copy(CODES)
    
    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab


def save_params(params):
    """
    Save parameters to file
    """
    pickle.dump(params, open('params.p', 'wb'))


def load_params():
    """
    Load parameters from file
    """
    return pickle.load(open('params.p', mode='rb'))


def batch_data(source, target, batch_size):
    """
    Batch source and target together
    """
    for batch_i in range(0, len(source)//batch_size):
        start_i = batch_i * batch_size
        source_batch = source[start_i:start_i + batch_size]
        target_batch = target[start_i:start_i + batch_size]
        yield np.array(pad_sentence_batch(source_batch)), np.array(pad_sentence_batch(target_batch))

def pad_sentence_batch(sentence_batch):
    """
    Pad sentence with </s> id
    """
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [CODES['</s>']] * (max_sentence - len(sentence))
            for sentence in sentence_batch]

def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_text_to_id = [[source_vocab_to_int[word] for word in line.split()] for line in source_text.split('\n')]
    target_text_to_id = [[target_vocab_to_int[word] for word in line.split()] for line in target_text.split('\n')]
    
    return (source_text_to_id, target_text_to_id)

In [2]:
import helper
import problem_unittests as tests
import tensorflow as tf

source_path = 'data/small_vocab_fr'
target_path = 'data/small_vocab_en'
source_text = load_data(source_path)
target_text = load_data(target_path)

In [3]:
preprocess_and_save_data(source_path, target_path)

{0: '<unk>', 1: '<s>', 2: '</s>', 3: 'france', 4: 'red', 5: 'mouse', 6: 'strawberries.', 7: 'least', 8: 'wants', 9: 'driving', 10: 'fun', 11: 'she', 12: 'february', 13: 'bananas', 14: 'lemon.', 15: 'strawberries', 16: 'apples', 17: 'apples.', 18: 'school', 19: 'feared', 20: 'peaches', 21: 'fruit.', 22: 'and', 23: 'sometimes', 24: 'india', 25: 'wet', 26: 'favorite.', 27: 'mice', 28: 'fruit', 29: 'mangoes', 30: 'spanish', 31: 'hot', 32: 'march', 33: 'bird', 34: 'black', 35: 'old', 36: 'dogs', 37: 'i', 38: 'chilly', 39: "isn't", 40: 'plan', 41: 'strawberry.', 42: 'lake', 43: 'her', 44: 'mangoes.', 45: 'orange.', 46: 'grape', 47: 'truck', 48: 'would', 49: 'french', 50: 'has', 51: 'want', 52: 'china', 53: 'was', 54: 'am', 55: 'mango.', 56: ',', 57: '?', 58: "it's", 59: 'thinks', 60: 'like', 61: 'bear', 62: 'lemon', 63: 'loved', 64: 'fall', 65: 'lemons.', 66: 'orange', 67: 'oranges.', 68: 'last', 69: 'horse', 70: 'easy', 71: 'october', 72: 'animals', 73: 'why', 74: 'do', 75: 'rainy', 76: 'to

In [4]:
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
# pad_sentence_batch(source_int_text)
source_vocab = len(source_vocab_to_int)
target_vocab = len(target_vocab_to_int)

In [5]:
source_int_text

[[49, 55, 306, 160, 168, 313, 61, 202, 281, 77, 307, 306, 209, 91, 346, 298],
 [262, 289, 306, 299, 105, 91, 173, 281, 77, 307, 127, 26, 91, 301, 298],
 [29, 306, 299, 168, 91, 100, 281, 77, 307, 306, 299, 85, 91, 227, 298],
 [262, 289, 306, 160, 337, 91, 227, 281, 77, 307, 327, 105, 91, 104, 298],
 [117, 250, 37, 14, 306, 58, 342, 281, 255, 97, 250, 37, 306, 54, 24, 298],
 [60, 14, 344, 306, 224, 281, 255, 97, 344, 306, 58, 342, 298],
 [349, 306, 142, 91, 131, 281, 255, 307, 306, 299, 105, 91, 173, 298],
 [49, 55, 306, 71, 42, 290, 281, 77, 307, 306, 283, 47, 91, 100, 298],
 [334, 14, 306, 250, 37, 58, 228, 281, 255, 97, 250, 37, 306, 58, 342, 298],
 [262, 289, 306, 160, 71, 91, 136, 281, 77, 307, 306, 160, 85, 91, 301, 298],
 [54, 215, 306, 60, 250, 37, 325, 274, 281, 255, 54, 130, 306, 97, 250, 10],
 [307, 332, 4, 212, 27, 118, 175, 298],
 [257, 306, 139, 91, 227, 281, 77, 307, 306, 160, 85, 91, 301, 298],
 [15, 200, 191, 97, 239, 58, 278, 37, 298],
 [307, 76, 324, 281, 259, 73, 77,

In [45]:
hidden_units = 256
n_layers = 1
num_encoder_symbols = source_vocab #source vocabulary size
num_decoder_symbols = target_vocab #target vocabulary size
learning_rate = 0.01
embedding_size = 15  #num of features for each word
max_gradient_norm = 5.0
dtype = tf.float32
# Number of Epochs
epochs = 10
# Batch Size
batch_size_train = 128
# Dropout Probability
dropout = 0.2
forget_bias = 1.0
use_beam_search = True
beam_width = 10
length_penalty_weight = 0.0
use_attention = True
learning_rate_decay = False

num_encoder_symbols, num_decoder_symbols

(357, 230)

In [46]:
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.contrib.rnn import MultiRNNCell
from tensorflow import layers

tf.reset_default_graph()

train_graph = tf.Graph()
with train_graph.as_default():
    
    ### DEFINING PLACEHOLDERS ###

    # encoder_inputs: [batch_size, max_time_steps]
    encoder_inputs = tf.placeholder(dtype=tf.int32,
                shape=(None, None), name='encoder_inputs')

    # encoder_inputs_length: [batch_size]
    encoder_inputs_length = tf.placeholder(
                dtype=tf.int32, shape=(None,), name='encoder_inputs_length')

    # get dynamic batch_size
    batch_size = tf.shape(encoder_inputs)[0]

    ### TRAIN MODE PLACEHOLDERS ###

    # decoder_inputs: [batch_size, max_time_steps]
    decoder_inputs = tf.placeholder(
                    dtype=tf.int32, shape=(None, None), name='decoder_inputs')

    # decoder_inputs_length: [batch_size]
    decoder_inputs_length = tf.placeholder(
                    dtype=tf.int32, shape=(None,), name='decoder_inputs_length')

    decoder_start_token = tf.ones(
                    shape=[batch_size, 1], dtype=tf.int32) * start_token
    decoder_end_token = tf.ones(
                    shape=[batch_size, 1], dtype=tf.int32) * end_token  


    # decoder_inputs_train: [batch_size , max_time_steps + 1]
    # insert sos symbol in front of each decoder input
    decoder_inputs_train = tf.concat([decoder_start_token,
                                          decoder_inputs], axis=1)

    # decoder_inputs_length_train: [batch_size]
    decoder_inputs_length_train = decoder_inputs_length + 1

    # decoder_targets_train: [batch_size, max_time_steps + 1]
    # insert eos symbol at the end of each decoder input
    decoder_targets_train = tf.concat([decoder_inputs,
                                           decoder_end_token], axis=1)

In [47]:
with train_graph.as_default():
    ## DEFINING ENCODER ##

    encoder_embeddings = tf.Variable(tf.random_uniform([num_encoder_symbols, embedding_size], -1.0, 1.0), dtype=dtype)

    # Embedded_inputs: [batch_size, time_step, embedding_size]
    encoder_inputs_embedded = tf.nn.embedding_lookup(
        params=encoder_embeddings, ids=encoder_inputs)
        
    # Build RNN cell
    cells = []
    for _ in range(n_layers):
        cell = tf.contrib.rnn.BasicLSTMCell(hidden_units, forget_bias=forget_bias)
        if dropout > 0.0:
            cell = tf.contrib.rnn.DropoutWrapper(
                cell=cell, input_keep_prob=(1.0 - dropout))
        cells.append(cell)
    if n_layers == 1:
        encoder_cells = cells[0]
    else:
        encoder_cells = tf.contrib.rnn.MultiRNNCell(cells)

    encoder_outputs, encoder_last_state = tf.nn.dynamic_rnn(
        cell=encoder_cells, inputs=encoder_inputs_embedded,
        sequence_length=encoder_inputs_length, dtype=dtype,
        time_major=False)

In [48]:
with train_graph.as_default():
    ### DEFINING DECODER ###

    
    # Building decoder_cell
    cells = []
    # Build RNN cell
    for _ in range(n_layers):
        cell = tf.contrib.rnn.BasicLSTMCell(hidden_units, forget_bias=forget_bias)
        if dropout > 0.0:
            cell = tf.contrib.rnn.DropoutWrapper(
                cell=cell, input_keep_prob=(1.0 - dropout))
        cells.append(cell)
    if n_layers == 1:
        decoder_cells = cells[0]
    else:
        decoder_cells = tf.contrib.rnn.MultiRNNCell(cells)

    if use_attention:
        memory = encoder_outputs
        
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
            hidden_units,
            memory,
            memory_sequence_length=encoder_inputs_length,
            normalize=True)
        
        decoder_cells_train = tf.contrib.seq2seq.AttentionWrapper(
            decoder_cells,
            attention_mechanism,
            attention_layer_size=hidden_units,
            alignment_history=False,
            output_attention=True,
            name="attention")
        
        decoder_initial_state = decoder_cells_train.zero_state(batch_size, dtype).clone(
          cell_state=encoder_last_state)
        
    else:
        decoder_cells_train = decoder_cells
        decoder_initial_state = encoder_last_state

    decoder_embeddings = tf.Variable(tf.random_uniform([num_decoder_symbols, embedding_size], -1.0, 1.0), dtype=dtype)
    
    # decoder_inputs_embedded: [batch_size, max_time_step + 1, embedding_size]
    decoder_inputs_embedded = tf.nn.embedding_lookup(
        params=decoder_embeddings, ids=decoder_inputs_train)

In [49]:
with train_graph.as_default():
    ### TRAIN MODE ###
    
    # Helper to feed inputs for training: read inputs from dense ground truth vectors
    training_helper = seq2seq.TrainingHelper(inputs=decoder_inputs_embedded,
                                       sequence_length=decoder_inputs_length_train,
                                       time_major=False,
                                        name='training_helper')

    training_decoder = seq2seq.BasicDecoder(cell=decoder_cells_train,
                                       helper=training_helper,
                                       initial_state=decoder_initial_state)

    # decoder_outputs_train: BasicDecoderOutput
    #                        namedtuple(rnn_outputs, sample_id)
    # decoder_outputs_train.rnn_output: [batch_size, max_time_step + 1, num_decoder_symbols] if output_time_major=False
    #                                   [max_time_step + 1, batch_size, num_decoder_symbols] if output_time_major=True
    # decoder_outputs_train.sample_id: [batch_size], tf.int32
    (decoder_outputs_train, decoder_last_state_train, 
         decoder_outputs_length_decode)  = seq2seq.dynamic_decode(decoder=training_decoder,
                                                        output_time_major=False,
                                                        impute_finished=True)

    # More efficient to do the projection on the batch-time-concatenated tensor
    # logits_train: [batch_size, max_time_step + 1, num_decoder_symbols]
    
    sample_id = decoder_outputs_train.sample_id
    
    output_layer = layers.Dense(num_decoder_symbols, name='output_projection')
    logits_train = output_layer(decoder_outputs_train.rnn_output)

In [50]:
with train_graph.as_default():
    
    ### LOSS, GRADIEND AND OPTIMIZATION ###
    
    if learning_rate_decay:
        global_step = tf.Variable(0, trainable=False)

        learning_rate = tf.constant(learning_rate)

        #using luong10 decay scheme
        decay_factor = 0.5
        start_decay_step = int(epochs / 2)
        decay_times = 10

        remain_steps = epochs - start_decay_step
        decay_steps = int(remain_steps / decay_times)

        learning_rate = tf.cond(global_step < start_decay_step,
                                lambda: learning_rate,
                                lambda: tf.train.exponential_decay(
                                    learning_rate,
                                    (global_step - start_decay_step),
                                    decay_steps, decay_factor, staircase=True),
                                name="learning_rate_decay_cond")
    
    # Maximum decoder time_steps in current batch
    max_decoder_length = tf.reduce_max(decoder_inputs_length_train)
    
    # masks: masking for valid and padded time steps, [batch_size, max_time_step + 1]
    target_weights = tf.sequence_mask(lengths=decoder_inputs_length_train, 
                             maxlen=max_decoder_length, dtype=dtype, name='masks')
    
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=decoder_targets_train, logits=logits_train)
    
    loss = (tf.reduce_sum(crossent * target_weights) /
        tf.cast(batch_size, dtype= dtype))

    trainable_params = tf.trainable_variables()
    
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    gradients = tf.gradients(loss, 
                             trainable_params)
    
    clip_gradients, gradient_norm = tf.clip_by_global_norm(gradients, max_gradient_norm)
    
    updates = opt.apply_gradients(
            zip(clip_gradients, trainable_params))
    

In [51]:
with train_graph.as_default():

    ### INFERENCE MODE ###
    
    start_tokens = tf.fill([batch_size], start_token)
    
    if use_attention:
        memory = tf.contrib.seq2seq.tile_batch(
          memory, multiplier=beam_width)
        
        source_sequence_length = tf.contrib.seq2seq.tile_batch(
          encoder_inputs_length, multiplier=beam_width)
        
        encoder_last_state = tf.contrib.seq2seq.tile_batch(
          encoder_last_state, multiplier=beam_width)
        
        batch_size = batch_size * beam_width
        
        attention_mechanism_infer = tf.contrib.seq2seq.BahdanauAttention(
            hidden_units,
            memory,
            memory_sequence_length=source_sequence_length,
            normalize=True)
        
        decoder_cells_infer = tf.contrib.seq2seq.AttentionWrapper(
            decoder_cells,
            attention_mechanism_infer,
            attention_layer_size=hidden_units,
            alignment_history=False,
            output_attention=True,
            name="attention_infer")
        
        decoder_initial_state_infer = decoder_cells_infer.zero_state(batch_size, dtype).clone(
          cell_state=encoder_last_state)
    
    if use_beam_search:
        #     decoder_initial_state_infer = tf.contrib.seq2seq.tile_batch(
        #           encoder_last_state, multiplier=beam_width)

        inference_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
              cell=decoder_cells_infer,
              embedding=decoder_embeddings,
              start_tokens=start_tokens,
              end_token=end_token,
              initial_state=decoder_initial_state_infer,
              beam_width=beam_width,
              output_layer=output_layer,
              length_penalty_weight=length_penalty_weight)
        
    else:
        inference_helper = seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                        start_tokens=start_tokens,
                                                        end_token=end_token)

        inference_decoder = seq2seq.BasicDecoder(cell=decoder_cells_infer,
                                                 helper=inference_helper,
                                                 initial_state=decoder_initial_state,
                                                 output_layer=output_layer)
    

    
    maximum_iterations = tf.round(tf.reduce_max(encoder_inputs_length) * 2)
    
    (decoder_infer_outputs, decoder_infer_last_state,
                 decoder_infer_outputs_length) = (seq2seq.dynamic_decode(
                    decoder=inference_decoder,
                    output_time_major=False,
                    maximum_iterations=maximum_iterations))
    
    if use_beam_search:
        decoder_pred_decode = decoder_infer_outputs.predicted_ids
    
    else:
        logits_infer = decoder_infer_outputs.rnn_output
        sample_id_infer = decoder_infer_outputs.sample_id

In [ ]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    lower_case_words = [word.lower() for word in sentence.split()]
    
    word_id = [vocab_to_int.get(word, vocab_to_int['<unk>']) for word in lower_case_words]
    
    return word_id

import time

### TRAINING ###
save_path = 'checkpoints/dev'

train_source = source_int_text[:]
train_target = target_int_text[:]

valid_source = source_int_text[:batch_size_train]
valid_target = target_int_text[:batch_size_train]

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                batch_data(train_source, train_target, batch_size_train)):
            start_time = time.time()
            
            source_batch_seq_lenght = []
            for item in source_batch:
                source_batch_seq_lenght.append(np.shape(item)[0])
            
            target_batch_seq_lenght = []
            for item in target_batch:
                target_batch_seq_lenght.append(np.shape(item)[0])
                
            print(source_batch_seq_lenght)
            print(target_batch_seq_lenght)
            
            if (source_batch_seq_lenght[0] > 300):
                continue
                
            _, loss_val = sess.run(
                [updates, loss],
                {encoder_inputs: source_batch,
                 decoder_inputs: target_batch,
                encoder_inputs_length: source_batch_seq_lenght,
                decoder_inputs_length: target_batch_seq_lenght})

            print('Epoch {:>3} Batch {:>4}/{}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size_train, loss_val))
            
            end_time = time.time()
            
    print("Training time: ", end_time - start_time)
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')
    
    translate_logits = sess.run(decoder_pred_decode, {encoder_inputs: [translate_sentence],
                                                       encoder_inputs_length: [np.shape(translate_sentence)[0]]})[0]

[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17

Epoch   0 Batch    9/1077, Loss: 73.628
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17

Epoch   0 Batch   18/1077, Loss: 42.720
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17

[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17

Epoch   0 Batch   37/1077, Loss: 18.303
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17

...

Epoch   3 Batch    9/1077, Loss: 1182.169
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 

Epoch   3 Batch   18/1077, Loss: 1168.852
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 

Epoch   3 Batch   29/1077, Loss: 1178.304
[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 

Epoch   3 Batch   38/1077, Loss: 1214.495
[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 

[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]
Epoch   3 Batch   46/1077, Loss: 1138.484
[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 

In [96]:
# Save parameters for checkpoint
save_params(save_path)

In [97]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()
load_path = load_params()

In [98]:
translate_sentence = "new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."
#fr to en
#input: "new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."
#target:"new jersey is sometimes quiet during autumn , and it is snowy in april ."

#en to vi
#input:  "Họ viết gần 1000 trang về chủ đề này ."
#target: "They wrote almost a thousand pages on the topic ."

print(translate_sentence)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)
print(np.shape(translate_sentence))

# loaded_graph = tf.Graph()
# with tf.Session(graph=loaded_graph) as sess:
#     # Load saved model
#     loader = tf.train.import_meta_graph(load_path + '.meta')
#     loader.restore(sess, load_path)

#     input_data = loaded_graph.get_tensor_by_name('input:0')
#     logits = loaded_graph.get_tensor_by_name('logits:0')
#     keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    translate_logits = sess.run(decoder_pred_decode, {encoder_inputs: [translate_sentence],
                                                       encoder_inputs_length: [np.shape(translate_sentence)[0]]})[0]
    
    print(translate_logits)

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i[0] for i in translate_logits]))
print('  French Words: {}'.format([target_int_to_vocab[i[0]] for i in translate_logits]))

print('Frech translation:')
translation = ''
for word_i in translate_logits:
    translation += target_int_to_vocab[word_i[0]] + ' '
    
print(translation)

They wrote almost a thousand pages on the topic .
(10,)


InvalidArgumentError: Input to reshape is a tensor with 256 values, but the requested shape has 2560
	 [[Node: Reshape = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](rnn/while/Exit_2, concat_3)]]
	 [[Node: decoder_1/transpose/_133 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_666_decoder_1/transpose", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Reshape', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflowGPU/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflowGPU/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflowGPU/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <modu